<a href="https://colab.research.google.com/github/gdabrow/Template/blob/master/analizy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Information on companies listed on stock exchanges obtained from:**
* https://seekingalpha.com/
* https://www.zacks.com/
* https://finance.yahoo.com/

In [ ]:
# installation libraries
! pip install yfinance --upgrade --no-cache-dir &> /dev/null
! pip install beautifulsoup4 &> /dev/null
! pip install cloudscraper &> /dev/null

In [ ]:
# import libraries
import yfinance as yf
import pandas as pd
from pandas.core.internals.base import DataManager
from bs4 import BeautifulSoup
import cloudscraper
import plotly.graph_objects as go
import plotly.express as px

scraper = cloudscraper.create_scraper()

# disk connection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**List with the data we are interested in**

In [ ]:
# columns = ['symbol','currentPrice','recommendationKey', 'recommendationMean', 'targetLowPrice', 'targetLowPrice [%]',
#          'targetMeanPrice', 'targetMeanPrice [%]', 'targetMedianPrice', 'targetMedianPrice [%]', 'targetHighPrice', 'targetHighPrice [%]',
#          'P/E', 'Forward P/E', 'EPS past 5Y', 'EPS (ttm)', 'forwardEps', 'EPS next Q', 'EPS next Y', 'EPS next 5Y', 'Profit Margin',
#           'ROA', 'ROE', 'ROI', 'currentRatio', 'quickRatio']

columns = ['symbol','currentPrice','recommendationKey', 'recommendationMean', 'targetLowPrice [%]',
           'targetMeanPrice [%]', 'targetMedianPrice [%]', 'targetHighPrice [%]',
           'P/E', 'EPS next Y', 'EPS next 5Y', 'Profit Margin',
           'ROA', 'ROE', 'ROI', 'currentRatio', 'quickRatio']

**Lists with tickers we are interested in**

In [ ]:
finansowa_edu = [ 'NFLX','MSFT','AAPL', 'NVDA','V','JNJ','AZN','SAP', 'RGEN','PG','AMT','GOOGL','AMZN']
Trela = ['TSM','INTU','NOW','PTC','ALGN','AMZN','QCOM', 'MA','ASML','MPWR','NVDA', 'AMD']
Kupione = ['EXAS', 'META', 'NIO', 'PDD', 'CRWD', 'MKSI', 'MDT', 'TSM', 'NFLX', 'MSFT', 'AAPL', 'V', 'BABA', 'TSM']
DNA = ['ROKU', 'TWLO', 'SE', 'DIS', 'UUUU', 'DNN']

**Tickers from seekingalpha if we want**

In [ ]:
# downloading data from a file on disk
seekingalpha = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Top.csv')
alpha = list(seekingalpha.Symbol)

In [ ]:

Tickers = alpha

In [ ]:
asset = pd.DataFrame(columns=columns)

for tic in Tickers: 

  url = 'https://finviz.com/quote.ashx?t=' + tic
  page = scraper.get(url)
  page_html = BeautifulSoup(page.content, 'html.parser')
  table = page_html.find_all('tr', class_='table-dark-row')
  
  if table == []:
    continue

  names = []
  value = []

  for row in table:
    for name in row.find_all('td', class_='snapshot-td2-cp'):
      names.append(name.text)

    for name in row.find_all('td', class_='snapshot-td2'):
      if name.text == '-':
        v = 0
      else:
        v = name.text
      value.append(v)

  finviz = dict(zip(names, value))

  data = yf.Ticker(tic).info
  data.update(finviz)

  try:
    data['targetLowPrice [%]'] = round(100 * float(data['targetLowPrice'])/float(data['currentPrice'])-100)
  except:
    data['targetLowPrice [%]'] = '-'

  try:
    data['targetMeanPrice [%]'] = round(100 * float(data['targetMeanPrice'])/float(data['currentPrice'])-100)
  except:
    data['targetMeanPrice [%]'] = '-'

  try:
    data['targetMedianPrice [%]'] = round(100 * float(data['targetMedianPrice'])/float(data['currentPrice'])-100)
  except:
    data['targetMedianPrice [%]'] = '-'

  try:
    data['targetHighPrice [%]'] = round(100 * float(data['targetHighPrice'])/float(data['currentPrice'])-100)
  except:
    data['targetHighPrice [%]'] = '-'

  row={}
  for col in columns:
    row[col] = data[col]

  asset = asset.append(row, ignore_index=True)


colum = asset.columns[9:-2].tolist()
#colum.remove('forwardEps')

for col in colum:
  asset[col] = asset[col].str.replace("%", "").astype(float)

asset.set_index('symbol', inplace=True)

In [ ]:
zacks = []

for tic in alpha:
  url = 'https://www.zacks.com/stock/quote/' + tic
  page = scraper.get(url)
  page_html = BeautifulSoup(page.content, 'html.parser')
  table = page_html.find_all('p', class_="rank_view")
  try:
    zac = list(table[0].get_text().replace(" ", ""))[1]
  except IndexError:
    zacks.append(0)
    continue

  zacks.append(zac)

In [ ]:
seekingalpha['zacks'] = zacks

In [ ]:
seekingalpha.drop(columns='Rank', inplace=True)

In [ ]:
asset.rename_axis("Symbol", inplace=True)

In [ ]:
seekingalpha.set_index('Symbol', inplace=True)

In [ ]:
test = pd.merge(left=seekingalpha, right=asset, on='Symbol')

In [ ]:
test

In [ ]:
def chart(symbol):

  tic = yf.Ticker(symbol)
  data = tic.history(period = "ytd")
  data['targetLowPrice'] = asset.loc[symbol, 'targetLowPrice']
  data['targetMeanPrice'] = asset.loc[symbol, 'targetMeanPrice']
  data['targetHighPrice'] = asset.loc[symbol, 'targetHighPrice']

  fig = go.Figure()

  fig.add_trace(go.Scatter(x=data.index, y=data.Close,
                             mode='lines',
                             name='Close'))
  
  fig.add_trace(go.Scatter(x=data.index, y=data['targetLowPrice'],
                             mode='lines',
                             name=asset.loc[symbol, 'targetLowPrice [%]']))
  
  fig.add_trace(go.Scatter(x=data.index, y=data['targetMeanPrice'],
                           mode='lines',
                           name=asset.loc[symbol, 'targetMeanPrice [%]']))

  fig.add_trace(go.Scatter(x=data.index, y=data['targetHighPrice'],
                           mode='lines',
                           name=asset.loc[symbol, 'targetHighPrice [%]']))
  fig.show()

In [ ]:
chart('BABA')

In [ ]:
fig = px.bar(asset, x=asset.index, y="recommendationMean", title='recommendation Mean')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='targetLowPrice [%]', title='target Low Price [%]')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='targetMeanPrice [%]', title='target Mean Price [%]')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='targetHighPrice [%]', title='target High Price [%]')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='P/E', title='P/E')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='Forward P/E', title='Forward P/E')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='EPS past 5Y', title='EPS past 5Y')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='EPS (ttm)', title='EPS (ttm)')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='forwardEps', title='forwardEps')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='EPS next Q', title='EPS next Q')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='EPS next Y', title='EPS next Y')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='EPS next 5Y', title='EPS next 5Y')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='Profit Margin', title='Profit Margin')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='ROA', title='ROA')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='ROE', title='ROE')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='ROI', title='ROI')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='currentRatio', title='currentRatio')
fig.show()

In [ ]:
fig = px.bar(asset, x=asset.index, y='quickRatio', title='quickRatio')
fig.show()

In [ ]:
tic = yf.Ticker('AMZN').financials
#tb = tb.transpose()

In [ ]:
tic

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Research Development,56052000000.0,42740000000.0,35931000000.0,28837000000.0
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,38155000000.0,24194000000.0,13962000000.0,11270000000.0
Minority Interest,None,None,None,None
Net Income,33364000000.0,21331000000.0,11588000000.0,10073000000.0
Selling General Administrative,116485000000.0,87193000000.0,64313000000.0,52177000000.0
Gross Profit,197478000000.0,152757000000.0,114986000000.0,93731000000.0
Ebit,24879000000.0,22899000000.0,14541000000.0,12421000000.0
Operating Income,24879000000.0,22899000000.0,14541000000.0,12421000000.0
Other Operating Expenses,62000000.0,-75000000.0,201000000.0,296000000.0


In [ ]:
fig = px.bar(tb, x=tb.index, y="Operating Income")
fig.show()